In [ ]:
import arxiv

In [ ]:
client = arxiv.Client()
topic = 'transformers stock'
search = arxiv.Search(
    query= topic,
    max_results=5,
    sort_by= arxiv.SortCriterion.Relevance
)

In [ ]:
papers = client.results(search=search)
for paper in papers:
    print(paper.title)

MASTER: Market-Guided Stock Transformer for Stock Price Forecasting
Meta-Stock: Task-Difficulty-Adaptive Meta-learning for Sub-new Stock Price Prediction
GRU-PFG: Extract Inter-Stock Correlation from Stock Factors with Graph Neural Network
An End-to-End Structure with Novel Position Mechanism and Improved EMD for Stock Forecasting
Pre-training Time Series Models with Stock Data Customization


In [ ]:
import os, json
from typing import List
PAPER_DIR = "./papers"
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    주제를 기반으로 arXiv에서 논문을 검색하고 해당 정보를 저장한다.

    Args:
        topic: 검색할 주제
        max_results: 검색할 최대 결과 수 (기본값: 5)

    Returns:
        검색에서 찾은 논문 ID 목록
    """
    # arxiv를 사용하여 논문 찾기
    client = arxiv.Client()
    search = arxiv.Search(
        query     = topic,
        max_results = max_results,
        sort_by   = arxiv.SortCriterion.Relevance
    )
    papers = client.results(search)

    # 이 주제에 대한 디렉토리 생성
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")

    # 기존 논문 정보 로드 시도
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # 각 논문을 처리하고 papers_info에 추가
    paper_ids = []
    for paper in papers:
        paper_id   = paper.get_short_id()
        paper_info = {
            "title"    : paper.title,
            "authors"  : [author.name for author in paper.authors],
            "summary"  : paper.summary,
            "pdf_url"  : paper.pdf_url,
            "published": str(paper.published.date())
        }
        paper_ids.append(paper_id)
        papers_info[paper_id] = paper_info

    # 업데이트된 papers_info를 json 파일에 저장
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"결과가 다음 위치에 저장되었습니다: {file_path}")
    return paper_ids

In [ ]:
search_papers("predict stock")

결과가 다음 위치에 저장되었습니다: ./papers\predict_stock\papers_info.json


['2002.05784v1',
 '2106.10159v1',
 '2308.11117v1',
 '1911.02194v1',
 '2411.18997v1']

In [ ]:
def extract_info(paper_id: str) -> str:
    """
    모든 주제 디렉토리에서 특정 논문에 관한 정보를 검색한다.

    Args:
        paper_id: 검색할 논문의 ID

    Returns:
        찾은 경우 논문 정보가 담긴 JSON 문자열, 찾지 못한 경우 오류 메시지
    """

    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"{file_path} 읽기 오류: {str(e)}")
                    continue

    return f"논문 {paper_id}와 관련된 저장된 정보가 없다."

In [ ]:
extract_info('20022d.05784v1')

'논문 20022d.05784v1와 관련된 저장된 정보가 없다.'

In [ ]:
functions = [
    {
        "name": "search_papers",
        "description": "주제를 기반으로 arXiv에서 논문을 검색하고 해당 정보를 저장한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "검색할 주제"
                },
                "max_results": {
                    "type": "integer",
                    "description": "검색할 최대 결과 수",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "모든 주제 디렉토리에서 특정 논문에 관한 정보를 검색한다.",
        "parameters": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "검색할 논문의 ID"
                }
            },
            "required": ["paper_id"]
        }
    }
]

In [ ]:
mapping_tool_function  = {
    "search_papers" : search_papers,
    "extract_info"  : extract_info
}

In [ ]:
def execute_tool(tool_name, tool_args):
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        return "작업이 완료되었찌만 결과는 반환하지 않았다"

    if isinstance(result, list):
        return ", ".join(result)

    if isinstance(result, dict):
        return json.dumps(result, indent=2, ensure_ascii=False)

    return str(result)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='')

In [ ]:
messages = [{'role' : 'user', 'content' : "최신 강화학습 트렌드"}]
resp  = client.chat.completions.create(
    model='gpt-4-0125-preview',
    messages=messages,
    functions=functions,
    function_call={"name": "search_papers"},
    max_tokens=2024
)

In [ ]:
msg = resp.choices[0].message

In [ ]:
msg

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"topic":"reinforcement learning","max_results":5}', name='search_papers'), tool_calls=None)

In [ ]:
call = msg.function_call

In [ ]:
call.name

'search_papers'

In [ ]:
json.loads(call.arguments)

{'topic': 'reinforcement learning', 'max_results': 5}

In [ ]:
tool_result = execute_tool(call.name,json.loads(call.arguments) )

결과가 다음 위치에 저장되었습니다: ./papers\reinforcement_learning\papers_info.json


In [ ]:
def process_query(query: str):
    messages = [{"role": "user", "content": query}]

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            functions=functions,
            function_call="auto",
            max_tokens=2024
        )
        msg = resp.choices[0].message

        # 1) 일반 텍스트 응답
        if msg.content is not None:
            print(msg.content)
            break

        # 2) 함수 호출 응답
        call = msg.function_call
        fname = call.name
        fargs = json.loads(call.arguments)

        print(f"Calling tool `{fname}` with args: {fargs}")
        tool_result = execute_tool(fname, fargs)

        # assistant의 함수 호출 메시지를 기록
        messages.append({
            "role": "assistant",
            "function_call": {
                "name": fname,
                "arguments": call.arguments
            }
        })
        # 함수 실행 결과를 function 역할로 추가
        messages.append({
            "role": "function",
            "name": fname,
            "content": tool_result
        })

In [ ]:
def chat_loop():
    print("쿼리를 입력하거나 'quit'를 입력해 종료합니다.")
    while True:
        query = input("Query: ").strip()
        if query.lower() == "quit":
            break
        try:
            process_query(query)

        except Exception as e:
            print(f"Error: {e}")

In [ ]:
chat_loop()

쿼리를 입력하거나 'quit'를 입력해 종료합니다.
Calling tool `search_papers` with args: {'topic': 'model context protocol'}
결과가 다음 위치에 저장되었습니다: ./papers\model_context_protocol\papers_info.json
Calling tool `extract_info` with args: {'paper_id': '2505.02279v2'}
Model Context Protocol (MCP)에 대한 정보는 다음과 같습니다:

### 제목
A survey of agent interoperability protocols: Model Context Protocol (MCP), Agent Communication Protocol (ACP), Agent-to-Agent Protocol (A2A), and Agent Network Protocol (ANP)

### 저자
- Abul Ehtesham
- Aditi Singh
- Gaurav Kumar Gupta
- Saket Kumar

### 요약
모델 기반 자동화 대리인은 툴 통합, 맥락 데이터 공유 및 이질적인 시스템 간 작업 조정을 위한 강력하고 표준화된 프로토콜을 요구합니다. 임의 통합은 확장 가능성이 낮고, 보안성이 떨어지며, 도메인 간 일반화하는 것이 어렵습니다. 이 조사에서는 배치 문맥에서 상호 운용성을 다루는 네 가지 신생 에이전트 통신 프로토콜인 Model Context Protocol (MCP), Agent Communication Protocol (ACP), Agent-to-Agent Protocol (A2A), Agent Network Protocol (ANP)를 검토합니다.

- **MCP**는 보안 도구 호출 및 타이핑된 데이터 교환을 위해 JSON-RPC 클라이언트-서버 인터페이스를 제공합니다.
- **ACP**는 RESTful HTTP를 통해 일반적인 통신 프로토콜을 정의하며, MIME 형식의 멀티파트 